# Rotating Drag Profile
This notebook is intended to connect with the xArm 7 and Bota Force/Torque Sensor. THen it will rotate the arm in set increments and record the drag force for a given duration. The forces and positions will be recorded as csv files in the data folder.

In [451]:
import os
from samplers import DataSource, PosSampler, FTSampler, Experiment
import threading
import time

from xarm.wrapper import XArmAPI
from util import rotate_forces
from scipy.signal import lfilter, lfilter_zi
import numpy as np

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from util import rotate_forces
from samplers import PosSampler, FTSampler, Experiment
from xarm.wrapper import XArmAPI
from scipy.signal import lfilter, lfilter_zi

SDK_VERSION: 1.13.30


Connect to the xArm and Bota force/torque sensor.

In [452]:
arm = XArmAPI('192.168.1.217')
arm.connect()

arm_sampler = PosSampler('arm', arm)
ft_sampler = FTSampler('FT', 'COM3')

ROBOT_IP: 192.168.1.217, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3


Create and experiment that will take care of sampling from the sensors and saving the data.

In [453]:
experiment = Experiment([arm_sampler, ft_sampler])
experiment.connect_sensors()

Opened serial port COM3
Trying to setup the sensor.
Timestep: 0.0011132812500000001
Frame synced
Collecting...
CRC mismatch received
Frame synced
CRC mismatch received
Frame synced
Collected for 10.000686168670654 seconds.


True

## Run an Experimental Trial
Change the values of the parameters to match the trial being conducted. The arm will move and the files will be named according to the values given.

In [515]:
arm.get_state()
arm.set_state(0)


0

### Move the Arm to the correct position for recording data

In [521]:
#_, angles = arm.get_servo_angle()
arm.set_servo_angle(angle=angles)

0

[SDK][ERROR][2024-08-12 05:50:46][base.py:247] - - [report-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host
[SDK][ERROR][2024-08-12 05:50:46][base.py:293] - - [main-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host


Exception in thread Thread-246 (_processdata_thread):
Traceback (most recent call last):
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\donatell\OneDrive - Chapman University\Teaching\CPSC230_Spring2024\Projects\Project Part 3\FHL-Projects\samplers.py", line 266, in _processdata_thread
    crc16_ccitt_frame = self._ser.read(2)
                        ^^^^^^^^^^^^^^^^^
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'Access is denied.', None, 5))


[SDK][ERROR][2024-08-12 05:50:47][decorator.py:39] - - xArm is not connected


Exception in thread Thread-245 (_processdata_thread):
Traceback (most recent call last):
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\donatell\OneDrive - Chapman University\Teaching\CPSC230_Spring2024\Projects\Project Part 3\FHL-Projects\samplers.py", line 63, in _processdata_thread
    state, (x, y, z, theta_x, theta_y, theta_z) = self.arm.get_position()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 6)


### Zero the sensor before turning on the flume

In [517]:
ft_sampler.zero()

Collecting...
Lost sync
Frame synced
Collected for 10.000279188156128 seconds.


In [520]:
# These values will control the experiment and be used to name the saveed data.
parameters = {
    'model': "DD11S",
    'angle': None,
    'flume-speed': 0.2
}
# sleep_duration should be long enough to allow the system to reach steady state before any data is recorded and duration is ho long you would like to record data for. Both times are in seconds.
sleep_duration = 10
duration = 4

arm.set_state(0)
_, (x, y, z, roll, pitch, yaw) = arm.get_position()
arm.set_position(x, y, z, roll, pitch, yaw)
for angle in [0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285, 300, 315, 330, 345, 360]:
    # The arm will move to the given location
    parameters['angle'] = angle
    arm.set_position(yaw=yaw+angle)

    # Sleep while the system reaches steady state
    time.sleep(sleep_duration)

    # Record Data
    print('Starting recording...')
    experiment.new_trial(parameters, duration=duration)
    print('Finished recording...')
    time.sleep(0.1)

    
    window_size = 450
    b = 1 / window_size * np.ones(window_size)
    a = 1
    zi = lfilter_zi(b, a)
    
    force_file = position_file = experiment.get_dir_from_parameters()
    force_file += '_FT/FT.csv'
    position_file += '_arm/arm.csv'
    df = rotate_forces(force_file, position_file)
    x, _ = lfilter(b, a, df['Fx'], zi=zi*df['Fx'].mean())
    y, _ = lfilter(b, a, df['Fy'], zi=zi*df['Fy'].mean())
    z, _ = lfilter(b, a, df['Fz'], zi=zi*df['Fz'].mean())
    df['Filtered x'] = x
    df['Filtered y'] = y
    df['Filtered z'] = z
    os.mkdir(experiment.get_dir_from_parameters()+'_rotated')
    df.to_csv(experiment.get_dir_from_parameters()+'_rotated/force.csv', index=False)
    
arm.set_position(yaw=yaw)

Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mismatch received
Frame synced
Starting recording...
Finished recording...
CRC mism

0

CRC mismatch received
Frame synced
